In [1]:
from sklearn.model_selection import cross_val_score, train_test_split
from tqdm import tqdm

import glob
import lightgbm as lgb
import numpy as np
import optuna as op
import os
import pandas as pd

op.logging.set_verbosity(op.logging.WARNING)

In [2]:
DATA_DIR = '/kaggle/input/march-machine-learning-mania-2024'

In [3]:
CSV = {}

for path in glob.glob(DATA_DIR + "/*.csv"):
    CSV[os.path.basename(path).split('.')[0]] = pd.read_csv(path, encoding='cp1252')

# Statistically based prediction

## T1 vs T2

In [4]:
def build_tx(gender):
    """
    Build the DF that includes T1 vs T2 and T2 vs T1 matchups.
    Concat two exact same DFs, but in one replace:
    - W => T1
    - L => T2
    and in the other one:
    - W => T2
    - L => T1
    """
    csv_names = ['NCAATourneyDetailedResults', 'RegularSeasonDetailedResults']
    csv_names = list(map(lambda x: gender + x, csv_names))
    csvs      = list(map(lambda x: CSV[x], csv_names))

    results_t1 = pd.concat(csvs)
    results_t1 = results_t1.drop(['DayNum', 'NumOT', 'WLoc'], axis=1)
    
    results_t2 = results_t1.copy()
        
    results_t1.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(results_t1.columns)]
    results_t2.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(results_t2.columns)]
    
    results = pd.concat([results_t1, results_t2]).reset_index(drop=True)
    results['ScoreDiff'] = results['T1_Score'] - results['T2_Score']
    results['Win'] = np.where(results['ScoreDiff'] > 0, 1, 0)

    return results

def build_t1_t2(gender):
    """
    Generate two DFs:
    - One preffixed by T1_
    - One preffixed by T2_
    """

    t1 = build_tx(gender)
    t1 = t1.drop(['T1_Score', 'T2_Score', 'T2_TeamID'], axis=1)
    t1 = t1.groupby(by=['Season', 'T1_TeamID']).mean()
    t2 = t1.copy()
    t2.index.names = ['Season', 'T2_TeamID']

    t1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") + "_Avg" for x in list(t1.columns)]
    t2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") + "_Avg" for x in list(t2.columns)]
    
    return (t1, t2)

In [5]:
def build_matchups(gender):
    """
    Generate a matchup DF. Each entry has matchups T1 vs each team (T2) as an Index in a 1-N relation.
    """
    teams = CSV["{}Teams".format(gender)]
    teams['T1_TeamID'] = teams['TeamID']
    teams['T2_TeamID'] = [teams['TeamID'].values.tolist() for i in teams.index]
    teams = teams.explode('T2_TeamID')
    teams = teams.groupby(['T1_TeamID', 'T2_TeamID']).sum()
    teams = teams.reset_index()
    teams = teams[['T1_TeamID', 'T2_TeamID']]
    teams = teams[teams['T1_TeamID'] != teams['T2_TeamID']]
    teams = teams.set_index(['T1_TeamID', 'T2_TeamID'])

    return teams

In [6]:
def build_fill(gender):
    """
    Build a DF with the same format as DF, used to fill NaN matchup values with overall AVGs.
    """
    matchups = build_matchups(gender)
    t1, t2   = build_t1_t2(gender)
    
    t1 = t1.groupby('T1_TeamID').mean()
    t2 = t2.groupby('T2_TeamID').mean()
    
    matchups = matchups.join(t1, on=['T1_TeamID'], how='inner')
    matchups = matchups.join(t2, on=['T2_TeamID'], how='inner')
    
    return matchups

In [7]:
def build_tourney(gender):
    """
    Generate a tourney DF. Each entry has matchups T1 vs each team (T2) as an Index in a 1-N relation.
    """
    tourney  = build_tx(gender)[['Season', 'T1_TeamID', 'T2_TeamID' , 'Win']]
    t1, t2   = build_t1_t2(gender)
    matchups = build_matchups(gender)
    fill     = build_fill(gender)
    
    tourney = pd.merge(matchups, tourney, on=['T1_TeamID', 'T2_TeamID'], how='left')
    tourney = pd.merge(tourney, t1, on=['Season', 'T1_TeamID'], how='left')
    tourney = pd.merge(tourney, t2, on=['Season', 'T2_TeamID'], how='left')
    tourney = tourney.groupby(by=['T1_TeamID', 'T2_TeamID']).mean()
    tourney = tourney.drop('Season', axis=1)
    tourney = tourney.fillna(fill)
    
    return tourney

In [8]:
def build_train_tourney(gender):
    """
    Build a train and tourney dataset, Train includes the Win column while Tourney may have NaNs.
    """
    tourney = build_tourney(gender)

    train = tourney[tourney['Win'].notnull()]
    train['Win'] = train['Win'].astype(int)
    
    return (train, tourney)

In [9]:
train_m, tourney_m = build_train_tourney('M')
train_w, tourney_w = build_train_tourney('W')

display(train_m)
display(tourney_m)
display(train_w)
display(tourney_w)

/tmp/ipykernel_18/3229804160.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Win'] = train['Win'].astype(int)
/tmp/ipykernel_18/3229804160.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Win'] = train['Win'].astype(int)


Win  T1_FGM_Avg  T1_FGA_Avg  T1_FGM3_Avg  T1_FGA3_Avg  \
T1_TeamID T2_TeamID                                                          
1101      1102         1   25.518519   56.481481     6.481481    19.962963   
          1115         1   21.428571   52.928571     7.250000    19.178571   
          1116         0   25.353636   59.185455     6.470909    18.569091   
          1117         0   25.292593   55.824074     6.824074    19.498148   
          1122         1   26.480000   59.280000     7.760000    21.320000   
...                  ...         ...         ...          ...          ...   
1478      1384         1   25.095238   59.238095     9.809524    27.428571   
          1437         0   25.095238   59.238095     9.809524    27.428571   
          1447         0   25.095238   59.238095     9.809524    27.428571   
          1467         0   25.095238   59.238095     9.809524    27.428571   
          1476         1   25.095238   59.238095     9.809524    27.428571   

                     T1_FTM_Avg  T1_FTA_Avg  T1_OR_Avg  T1_DR_Avg  T1_Ast_Avg  \
T1_TeamID T2_TeamID                                                             
1101      1102        13.148148   18.666667   9.037037  23.222222   13.888889   
          1115        10.892857   14.964286   8.250000  19.642857   11.857143   
          1116        15.126364   21.245455   8.797273  22.140909   14.390909   
          1117        13.324074   18.716667   8.951852  22.911111   14.161111   
          1122        13.480000   19.400000   9.640000  22.600000   17.600000   
...                         ...         ...        ...        ...         ...   
1478      1384        10.761905   13.904762   7.380952  21.857143   15.142857   
          1437        10.761905   13.904762   7.380952  21.857143   15.142857   
          1447        10.761905   13.904762   7.380952  21.857143   15.142857   
          1467        10.761905   13.904762   7.380952  21.857143   15.142857   
          1476        10.761905   13.904762   7.380952  21.857143   15.142857   

                     ...  T2_opponent_FTA_Avg  T2_opponent_OR_Avg  \
T1_TeamID T2_TeamID  ...                                            
1101      1102       ...            18.137931            9.172414   
          1115       ...            24.156250           12.250000   
          1116       ...            19.875000            8.531250   
          1117       ...            22.833333           10.300000   
          1122       ...            17.800000            8.080000   
...                  ...                  ...                 ...   
1478      1384       ...            19.590909            9.363636   
          1437       ...            15.791667            6.875000   
          1447       ...            15.600000            7.800000   
          1467       ...            18.913043           11.304348   
          1476       ...            18.520000           10.200000   

                     T2_opponent_DR_Avg  T2_opponent_Ast_Avg  \
T1_TeamID T2_TeamID                                            
1101      1102                26.034483            14.586207   
          1115                23.250000            13.000000   
          1116                23.895833            12.703125   
          1117                25.516667            14.050000   
          1122                21.520000            11.800000   
...                                 ...                  ...   
1478      1384                22.863636            13.681818   
          1437                23.291667            13.166667   
          1447                24.700000            14.050000   
          1467                24.478261            13.478261   
          1476                26.960000            17.120000   

                     T2_opponent_TO_Avg  T2_opponent_Stl_Avg  \
T1_TeamID T2_TeamID                                            
1101      1102                13.965517             5.586207   
          1115               

Win  T1_FGM_Avg  T1_FGA_Avg  T1_FGM3_Avg  T1_FGA3_Avg  \
T1_TeamID T2_TeamID                                                          
1101      1102       1.0   25.518519   56.481481     6.481481    19.962963   
          1103       NaN   24.192179   55.786654     6.754721    19.318635   
          1104       NaN   24.192179   55.786654     6.754721    19.318635   
          1105       NaN   24.192179   55.786654     6.754721    19.318635   
          1106       NaN   24.192179   55.786654     6.754721    19.318635   
...                  ...         ...         ...          ...          ...   
1478      1473       NaN   25.095238   59.238095     9.809524    27.428571   
          1474       NaN   25.095238   59.238095     9.809524    27.428571   
          1475       NaN   25.095238   59.238095     9.809524    27.428571   
          1476       1.0   25.095238   59.238095     9.809524    27.428571   
          1477       NaN   25.095238   59.238095     9.809524    27.428571   

                     T1_FTM_Avg  T1_FTA_Avg  T1_OR_Avg  T1_DR_Avg  T1_Ast_Avg  \
T1_TeamID T2_TeamID                                                             
1101      1102        13.148148   18.666667   9.037037  23.222222   13.888889   
          1103        14.414827   20.083464   8.536771  21.543447   13.443715   
          1104        14.414827   20.083464   8.536771  21.543447   13.443715   
          1105        14.414827   20.083464   8.536771  21.543447   13.443715   
          1106        14.414827   20.083464   8.536771  21.543447   13.443715   
...                         ...         ...        ...        ...         ...   
1478      1473        10.761905   13.904762   7.380952  21.857143   15.142857   
          1474        10.761905   13.904762   7.380952  21.857143   15.142857   
          1475        10.761905   13.904762   7.380952  21.857143   15.142857   
          1476        10.761905   13.904762   7.380952  21.857143   15.142857   
          1477        10.761905   13.904762   7.380952  21.857143   15.142857   

                     ...  T2_opponent_FTA_Avg  T2_opponent_OR_Avg  \
T1_TeamID T2_TeamID  ...                                            
1101      1102       ...            18.137931            9.172414   
          1103       ...            19.477682           10.811157   
          1104       ...            19.628514           11.391395   
          1105       ...            22.729740           12.171504   
          1106       ...            22.405766           10.944926   
...                  ...                  ...                 ...   
1478      1473       ...            20.475705            8.764107   
          1474       ...            17.453333            8.690000   
          1475       ...            25.126959            7.798589   
          1476       ...            18.520000           10.200000   
          1477       ...            22.366477            8.698864   

                     T2_opponent_DR_Avg  T2_opponent_Ast_Avg  \
T1_TeamID T2_TeamID                                            
1101      1102                26.034483            14.586207   
          1103                22.914088            11.718818   
          1104                22.776503            11.733135   
          1105                25.796000            13.277825   
          1106                24.684299            12.345102   
...                                 ...                  ...   
1478      1473                25.474138            13.789185   
          1474                24.883333            13.293333   
          1475                23.658307            12.538401   
          1476                26.960000            17.120000   
          1477                24.931818            13.197443   

                     T2_opponent_TO_Avg  T2_opponent_Stl_Avg  \
T1_TeamID T2_TeamID                                            
1101      1102                13.965517             5.586207   
          1103               

Win  T1_FGM_Avg  T1_FGA_Avg  T1_FGM3_Avg  T1_FGA3_Avg  \
T1_TeamID T2_TeamID                                                          
3101      3102         1   21.720000   57.600000     7.280000    26.040000   
          3106         1   25.555556   58.000000     8.296296    24.444444   
          3114         1   24.269231   57.423077     8.730769    25.076923   
          3116         0   23.893899   54.519231     8.144562    24.625332   
          3117         1   24.269231   57.423077     8.730769    25.076923   
...                  ...         ...         ...          ...          ...   
3478      3425         0   20.260870   57.000000     4.913043    17.347826   
          3433         0   20.260870   57.000000     4.913043    17.347826   
          3447         1   20.260870   57.000000     4.913043    17.347826   
          3467         1   20.260870   57.000000     4.913043    17.347826   
          3476         1   20.260870   57.000000     4.913043    17.347826   

                     T1_FTM_Avg  T1_FTA_Avg  T1_OR_Avg  T1_DR_Avg  T1_Ast_Avg  \
T1_TeamID T2_TeamID                                                             
3101      3102        14.320000   20.440000  15.320000  24.560000   13.040000   
          3106        10.444444   15.111111   9.666667  22.000000   13.925926   
          3114        13.653846   18.192308   9.653846  20.076923   15.076923   
          3116        12.529178   16.830239  11.941645  23.077586   14.740053   
          3117        13.653846   18.192308   9.653846  20.076923   15.076923   
...                         ...         ...        ...        ...         ...   
3478      3425        10.347826   13.913043   9.739130  20.086957   11.782609   
          3433        10.347826   13.913043   9.739130  20.086957   11.782609   
          3447        10.347826   13.913043   9.739130  20.086957   11.782609   
          3467        10.347826   13.913043   9.739130  20.086957   11.782609   
          3476        10.347826   13.913043   9.739130  20.086957   11.782609   

                     ...  T2_opponent_FTA_Avg  T2_opponent_OR_Avg  \
T1_TeamID T2_TeamID  ...                                            
3101      3102       ...            21.933333           16.000000   
          3106       ...            20.033333           12.233333   
          3114       ...            18.360000            8.480000   
          3116       ...            17.360057           13.474383   
          3117       ...            21.200000           11.000000   
...                  ...                  ...                 ...   
3478      3425       ...            14.818182            7.818182   
          3433       ...            13.166667            6.833333   
          3447       ...            18.850000           11.550000   
          3467       ...            18.086957           10.000000   
          3476       ...            14.347826           11.434783   

                     T2_opponent_DR_Avg  T2_opponent_Ast_Avg  \
T1_TeamID T2_TeamID                                            
3101      3102                31.666667            17.666667   
          3106                21.700000            12.500000   
          3114                23.000000             9.760000   
          3116                30.582068            14.612903   
          3117                25.520000            15.680000   
...                                 ...                  ...   
3478      3425                22.227273            12.500000   
          3433                18.750000             9.166667   
          3447                23.700000            15.600000   
          3467                22.347826            12.043478   
          3476                24.739130            14.217391   

                     T2_opponent_TO_Avg  T2_opponent_Stl_Avg  \
T1_TeamID T2_TeamID                                            
3101      3102                14.400000             9.133333   
          3106               

Win  T1_FGM_Avg  T1_FGA_Avg  T1_FGM3_Avg  T1_FGA3_Avg  \
T1_TeamID T2_TeamID                                                          
3101      3102       1.0   21.720000   57.600000     7.280000    26.040000   
          3103       NaN   24.106831   56.689811     8.243338    25.274662   
          3104       NaN   24.106831   56.689811     8.243338    25.274662   
          3105       NaN   24.106831   56.689811     8.243338    25.274662   
          3106       1.0   25.555556   58.000000     8.296296    24.444444   
...                  ...         ...         ...          ...          ...   
3478      3473       NaN   20.260870   57.000000     4.913043    17.347826   
          3474       NaN   20.260870   57.000000     4.913043    17.347826   
          3475       NaN   20.260870   57.000000     4.913043    17.347826   
          3476       1.0   20.260870   57.000000     4.913043    17.347826   
          3477       NaN   20.260870   57.000000     4.913043    17.347826   

                     T1_FTM_Avg  T1_FTA_Avg  T1_OR_Avg  T1_DR_Avg  T1_Ast_Avg  \
T1_TeamID T2_TeamID                                                             
3101      3102        14.320000   20.440000  15.320000  24.560000   13.040000   
          3103        13.325945   18.507925  11.928763  23.537225   14.138603   
          3104        13.325945   18.507925  11.928763  23.537225   14.138603   
          3105        13.325945   18.507925  11.928763  23.537225   14.138603   
          3106        10.444444   15.111111   9.666667  22.000000   13.925926   
...                         ...         ...        ...        ...         ...   
3478      3473        10.347826   13.913043   9.739130  20.086957   11.782609   
          3474        10.347826   13.913043   9.739130  20.086957   11.782609   
          3475        10.347826   13.913043   9.739130  20.086957   11.782609   
          3476        10.347826   13.913043   9.739130  20.086957   11.782609   
          3477        10.347826   13.913043   9.739130  20.086957   11.782609   

                     ...  T2_opponent_FTA_Avg  T2_opponent_OR_Avg  \
T1_TeamID T2_TeamID  ...                                            
3101      3102       ...            21.933333           16.000000   
          3103       ...            18.244050           12.255627   
          3104       ...            17.449215           12.279338   
          3105       ...            20.983834           12.532968   
          3106       ...            20.033333           12.233333   
...                  ...                  ...                 ...   
3478      3473       ...            18.547619           10.081905   
          3474       ...            25.140110            9.640110   
          3475       ...            19.873077            9.759615   
          3476       ...            14.347826           11.434783   
          3477       ...            16.937500           10.746875   

                     T2_opponent_DR_Avg  T2_opponent_Ast_Avg  \
T1_TeamID T2_TeamID                                            
3101      3102                31.666667            17.666667   
          3103                25.311055            13.357218   
          3104                25.188613            12.507329   
          3105                24.624642            12.461237   
          3106                21.700000            12.500000   
...                                 ...                  ...   
3478      3473                23.400952            14.202857   
          3474                26.595238            13.891026   
          3475                22.401923            12.153846   
          3476                24.739130            14.217391   
          3477                27.203125            14.643750   

                     T2_opponent_TO_Avg  T2_opponent_Stl_Avg  \
T1_TeamID T2_TeamID                                            
3101      3102                14.400000             9.133333   
          3103               

# Model building

In [10]:
def score_dataset(lgbm_params, X, y):
    classifier = lgb.LGBMClassifier(**lgbm_params, class_weight='balanced')
    score      = cross_val_score(classifier, X, y, scoring='neg_brier_score')
    score      = -1 * score.mean() + score.std()

    return score

def objective(trial, X, y):
    params = {
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves': trial.suggest_int('num_leaves', 5, 31),
        'n_estimators': trial.suggest_int('n_estimators', 1, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 300),
        'device_type': 'cpu',
        'verbose': -1
    }

    return score_dataset(params, X, y)

def study(X, y):
    study = op.create_study()
    study.optimize(lambda trial: objective(trial, X, y), n_trials=50, show_progress_bar=True)

    return study.best_params

In [11]:
def build_x_y(df):
    target_column = 'Win'
    feature_columns = df.columns.tolist()
    feature_columns.remove(target_column)
    
    return df[feature_columns], df[target_column]

In [12]:
X_m, y_m = build_x_y(train_m)
X_w, y_w = build_x_y(train_w)

In [13]:
params_m = study(X_m, y_m)
params_w = study(X_w, y_w)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
def accuracy(X, y, params):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    clf_test = lgb.LGBMClassifier(**params, class_weight='balanced')
    clf_test.fit(X_train, y_train)

    print('LightGBM Model accuracy score: {0:0.4f}'.format(clf_test.score(X_test, y_test)))
    print('LightGBM Model accuracy score [train]: {0:0.4f}'.format(clf_test.score(X_train, y_train)))

In [15]:
accuracy(X_m, y_m, params_m)
accuracy(X_w, y_w, params_w)

[LightGBM] [Info] Number of positive: 10860, number of negative: 20967
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.039697 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 31827, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
LightGBM Model accuracy score: 0.7800
LightGBM Model accuracy score [train]: 0.7961
[LightGBM] [Info] Number of positive: 7237, number of negative: 14873
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 22110, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> i

# Prediction

In [16]:
def build_proba(X, y, tourney, params):
    clf = lgb.LGBMClassifier(**params, class_weight='balanced')
    clf.fit(X, y)
    
    tourney = tourney.drop('Win', axis=1)
    results = tourney

    pred = clf.predict_proba(results)
    results['Probability'] = list(map(lambda x: x[1], pred))

    return results['Probability']

In [17]:
proba_m = build_proba(X_m, y_m, tourney_m, params_m)
proba_w = build_proba(X_w, y_w, tourney_w, params_w)

display(proba_m)
display(proba_w)

[LightGBM] [Info] Number of positive: 15410, number of negative: 30058
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040805 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 45468, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Info] Number of positive: 10375, number of negative: 21211
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036227 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 31586, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


T1_TeamID  T2_TeamID
1101       1102         0.436352
           1103         0.149712
           1104         0.144192
           1105         0.482132
           1106         0.274666
                          ...   
1478       1473         0.662140
           1474         0.513471
           1475         0.561145
           1476         0.784775
           1477         0.606464
Name: Probability, Length: 142506, dtype: float64

T1_TeamID  T2_TeamID
3101       3102         0.827413
           3103         0.324658
           3104         0.288112
           3105         0.627772
           3106         0.695935
                          ...   
3478       3473         0.735133
           3474         0.710742
           3475         0.317290
           3476         0.802926
           3477         0.346844
Name: Probability, Length: 141000, dtype: float64

In [18]:
def build_slots(gender):
    slots = CSV["{}NCAATourneySlots".format(gender)]
    slots = slots[slots['Season'] == 2023]
    slots = slots[slots['Slot'].str.contains('R')] 

    return slots

In [19]:
slots_m = build_slots('M')
slots_w = build_slots('W')

display(slots_m)
display(slots_w)

,Season,Slot,StrongSeed,WeakSeed
2385,2023,R1W1,W01,W16
2386,2023,R1W2,W02,W15
2387,2023,R1W3,W03,W14
2388,2023,R1W4,W04,W13
2389,2023,R1W5,W05,W12
...,...,...,...,...
2443,2023,R4Y1,R3Y1,R3Y2
2444,2023,R4Z1,R3Z1,R3Z2
2445,2023,R5WX,R4W1,R4X1
2446,2023,R5YZ,R4Y1,R4Z1


,Season,Slot,StrongSeed,WeakSeed
1579,2023,R1W1,W01,W16
1580,2023,R1W2,W02,W15
1581,2023,R1W3,W03,W14
1582,2023,R1W4,W04,W13
1583,2023,R1W5,W05,W12
...,...,...,...,...
1637,2023,R4Y1,R3Y1,R3Y2
1638,2023,R4Z1,R3Z1,R3Z2
1639,2023,R5WX,R4W1,R4X1
1640,2023,R5YZ,R4Y1,R4Z1


In [20]:
def build_seeds_2024():
    seeds_2024 = CSV['2024_tourney_seeds']

    return seeds_2024[seeds_2024['Tournament'] == 'M'], seeds_2024[seeds_2024['Tournament'] == 'W']

In [21]:
seeds_2024_m, seeds_2024_w = build_seeds_2024()

display(seeds_2024_m)
display(seeds_2024_w)

,Tournament,Seed,TeamID
0,M,W01,1345
1,M,W02,1266
2,M,W03,1243
3,M,W04,1397
4,M,W05,1181
...,...,...,...
59,M,Z12,1433
60,M,Z13,1233
61,M,Z14,1213
62,M,Z15,1421


,Tournament,Seed,TeamID
64,W,W01,3376
65,W,W02,3268
66,W,W03,3323
67,W,W04,3417
68,W,W05,3328
...,...,...,...
123,W,Z12,3405
124,W,Z13,3387
125,W,Z14,3241
126,W,Z15,3436


In [22]:
def prepare_data(seeds):
    seed_dict = seeds.set_index('Seed')['TeamID'].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}

    return seed_dict, inverted_seed_dict


def simulate(round_slots, seeds, inverted_seeds, proba):
    '''
    Simulates each round of the tournament.

    Parameters:
    - round_slots: DataFrame containing information on who is playing in each round.
    - seeds (dict): Dictionary mapping seed values to team IDs.
    - inverted_seeds (dict): Dictionary mapping team IDs to seed values.
    - proba (DF): DF that includes wins prediction per matchup.
    Returns:
    - list: List with winning team IDs for each match.
    - list: List with corresponding slot names for each match.
    '''
    winners = []
    slots = []

    for slot, strong, weak in zip(round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed):
        team_1, team_2 = seeds[strong], seeds[weak]
        
        team_1_prob = proba.loc[team_1, team_2]
        winner = np.random.choice([team_1, team_2], p=[team_1_prob, 1 - team_1_prob])

        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    return [inverted_seeds[w] for w in winners], slots


def run_simulation(seeds, round_slots, proba, brackets):
    '''
    Runs a simulation of bracket tournaments.

    Parameters:
    - seeds (pd.DataFrame): DataFrame containing seed information.
    - round_slots (pd.DataFrame): DataFrame containing information about the tournament rounds.
    - wins (DF): DF that includes wins prediction per matchup.
    - brackets (int): Number of brackets to simulate.
    Returns:
    - pd.DataFrame: DataFrame with simulation results.
    '''
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict = prepare_data(seeds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets + 1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, proba)
        
        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({'Bracket': bracket, 'Slot': slots, 'Team': results})

    return result_df

In [23]:
n_brackets = 50000
result_m = run_simulation(seeds_2024_m, slots_m, proba_m, n_brackets)
result_m.insert(0, 'Tournament', 'M')
result_w = run_simulation(seeds_2024_w, slots_w, proba_w, n_brackets)
result_w.insert(0, 'Tournament', 'W')

100%|██████████| 50000/50000 [04:48<00:00, 173.32it/s]


In [24]:
submission = pd.concat([result_m, result_w])
submission.reset_index(inplace=True, drop=True)
submission.index.names = ['RowId']

submission

,Tournament,Bracket,Slot,Team
RowId,,,,
0,M,1,R1W1,W01
1,M,1,R1W2,W02
2,M,1,R1W3,W03
3,M,1,R1W4,W04
4,M,1,R1W5,W12
...,...,...,...,...
6299995,W,50000,R4Y1,Y03
6299996,W,50000,R4Z1,Z09
6299997,W,50000,R5WX,X05


In [25]:
submission.to_csv('submission.csv')

# Resources
- https://www.kaggle.com/code/toshimelonhead/ncaa-march-madness-sabermetric-spin-v2
- https://www.kaggle.com/code/rustyb/paris-madness-2023
- Simulation based on: https://www.kaggle.com/code/lennarthaupts/simulate-n-brackets by _Lennart Haupts_.